In [91]:
#-*- coding : utf-8 -*-
# coding: utf-8
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import keras
from keras.callbacks import LambdaCallback
import random

## 加载数据

In [92]:
text = ''
i = 0
with open('Poem.txt', 'r', encoding='gb18030') as fr:   # gb18030兼容中文比较好
    for line in fr:
        if '第' in line or '◎' in line:  # 删除冗余信息，只保留古诗内容。
            text += '+'
        else:
            text += line.strip()
        i += 1
#         if i == 50:  # 为了训练速度，只选择前2000行诗歌作为样本训练
#             break


poems = text.split('+')  # 诗歌列表
text = ''

five_len_poems = []   # 五言绝句
for p in poems:
    if len(p) > 7 and p[5] == '，':  # 判断“，”是否中文的逗号！！！！
        five_len_poems.append(p)
        p += '+'
        text += p  # 重构text文本
print(len(poems))
print(len(five_len_poems))
# print(five_len_poems[:1000])
# print(text[:10000])

44640
24087


## 创建字典

In [93]:
words = sorted(list(text))

counted_words = {}
for word in words:
    if word in counted_words:
        counted_words[word] += 1
    else:
        counted_words[word] = 1
# 去掉低频的字
erase = []
for key in counted_words:
    if counted_words[key] <= 3:
        erase.append(key)
for key in erase:
    del counted_words[key]
wordPairs = sorted(counted_words.items(), key=lambda x: -x[1])
 
words, _ = zip(*wordPairs)
words += (" ",)

# word到id的映射
word2num = dict((c, i) for i, c in enumerate(words))
num2word = dict((i, c) for i, c in enumerate(words))
word2numF = lambda x: word2num.get(x, len(words) - 1)  # 返回索引的函数
print(words)
print(type(word2numF))

('，', '。', '+', '不', '人', '山', '无', '日', '风', '云', '一', '何', '有', '天', '中', '来', '水', '时', '月', '上', '为', '春', '生', '心', '相', '自', '花', '此', '君', '长', '秋', '如', '归', '白', '知', '清', '行', '见', '年', '空', '去', '在', '高', '江', '下', '远', '夜', '客', '未', '多', '寒', '里', '明', '门', '子', '得', '落', '道', '青', '路', '出', '我', '朝', '雨', '与', '事', '思', '草', '南', '入', '千', '处', '金', '深', '声', '树', '流', '城', '已', '三', '今', '地', '可', '色', '万', '新', '别', '飞', '东', '玉', '家', '还', '林', '开', '独', '谁', '是', '前', '复', '石', '同', '烟', '游', '望', '光', '将', '尽', '闻', '马', '欲', '回', '海', '成', '西', '酒', '方', '古', '应', '雪', '重', '闲', '从', '阳', '难', '书', '言', '情', '身', '气', '老', '鸟', '衣', '亦', '非', '更', '愁', '尘', '向', '发', '名', '所', '能', '意', '然', '当', '-', '香', '分', '叶', '看', '歌', '旧', '满', '松', '北', '平', '犹', '幽', '离', '起', '外', '故', '过', '孤', '作', '暮', '竹', '安', '随', '后', '诗', '华', '黄', '大', '晚', '龙', '坐', '野', '临', '间', '阴', '终', '初', '国', '吟', '文', '露', '楼', '几', '莫', '台', '岁', '公', '岂', '芳', '仙', '馀', '百', '似', '期',

## 构造训练训练数据

In [94]:
seq_length = 6  # 取前六个文字，预测第七个文字
def data_generator():
    i = 0
    while 1:
        sequence = text[i:i + seq_length]
        label =text[i + seq_length]
        if '+' not in sequence and label != '+' :  # 存在‘+’说明是两首诗，舍弃该数据即可。
            y_vec = np.zeros(
                shape=(1, len(words)),
                dtype=np.bool
            )
            y_vec[0, word2numF(label)] = 1.0

            x_vec = np.zeros(
                shape=(1, seq_length, len(words)),  # 三维矩阵(1, 5, len(words))
                dtype=np.bool
            )

            for t, char in enumerate(sequence):
                x_vec[0, t, word2numF(char)] = 1.0
            yield x_vec, y_vec
        i+=1
        

## LSTM建模

In [95]:
model = Sequential()
model.add(LSTM(512, input_shape=(seq_length, len(words)), return_sequences=True))
# 为了使下一个LSTM层能够处理相同的序列，我们输入return_sequence参数为真
model.add(Dropout(0.6))
model.add(LSTM(256))  
model.add(Dropout(0.6))
model.add(Dense(len(words), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])

## 训练过程中，每个epoch打印出当前的学习情况

In [96]:
def sample(preds, temperature=1.0):
    '''
    当temperature=1.0时，模型输出正常
    当temperature=0.5时，模型输出比较open
    当temperature=1.5时，模型输出比较保守
    在训练的过程中可以看到temperature不同，结果也不同
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [97]:
def generate_sample_result(epoch, logs):
    '''
    训练过程中，
    每个epoch打印出当前的学习情况
    '''
    print("\n==================Epoch {%s}=====================" % str(epoch))
    for diversity in [0.7, 1.2, 1.6]:
        print("------------Diversity {%s}--------------" % str(diversity))
        generated = ''
        
        # 随机选取一句开头的诗歌，生成五言绝句
        poem_index = random.randint(0, len(five_len_poems)-1)
        sentence = five_len_poems[poem_index][:seq_length]
        
        generated += sentence
        for i in range(18):
            x_pred = np.zeros((1, seq_length, len(words)))
            for t, char in enumerate(sentence):
                x_pred[0, t, word2numF(char)] = 1.0

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = num2word[next_index]
          
            generated += next_char
            sentence = sentence + next_char[1:]  # 取后五个字，进行下一轮的预测
        print(generated)

## 训练模型

In [98]:
weight_file = 'poetry_model.h5'
batch_size = 128
number_of_epoch = int(len(text) // batch_size / 2.5)
history = model.fit_generator(
    generator=data_generator(),
    verbose=1,
    steps_per_epoch=batch_size,
    epochs=number_of_epoch,
    callbacks=[
            keras.callbacks.ModelCheckpoint(weight_file, save_weights_only=False),
            LambdaCallback(on_epoch_end=generate_sample_result)  # 训练过程中，打印模型生成诗歌的效果
            ]
)

Epoch 1/5816
128/128 [==============================] - 53s 415ms/step - loss: 8.0241 - acc: 0.0781

==================Epoch {0}=====================
------------Diversity {0.7}--------------
去年潼关破，嫁屩饬镐历汩翎欠跂能奚渭放罕但斫没橦
------------Diversity {1.2}--------------
万里三韩国，惚躞粲踌嗽踠忏霆坡利霄睥下肆锧刍蹲戎
------------Diversity {1.6}--------------
尝读高士传，嗈架荥肇脯铁载冽风玖姮铩掣雨詹虾展遂
Epoch 2/5816
128/128 [==============================] - 45s 350ms/step - loss: 7.3654 - acc: 0.0703

==================Epoch {1}=====================
------------Diversity {0.7}--------------
白发轮台使，隗虚缝赏篁旦惋识烟认掀听彩洧彩烟迥草
------------Diversity {1.2}--------------
高龛险欲摧，。玲巷府继穆致邓撑管差蝼虚薨盖彴孟肆
------------Diversity {1.6}--------------
公府日无事，栅i抗褕坐纱可沟动肩爵燃儒同笺轹酬藤
Epoch 3/5816
128/128 [==============================] - 45s 352ms/step - loss: 7.4389 - acc: 0.1250

==================Epoch {2}=====================
------------Diversity {0.7}--------------
诸侯崇鲁学，。赏，可。赏去。。欢。釭飐硎。去壖云
------------Diversity {1.2}--------------
养疾衡檐下，猱沙落锸纮虓城壖绮妇e聊罗供类呷岐z
------------Di

128/128 [==============================] - 45s 350ms/step - loss: 6.5717 - acc: 0.1562

==================Epoch {23}=====================
------------Diversity {0.7}--------------
荒城自萧索，阴章忘宜卫凤幌兵堤轩临池出玄游阙席水
------------Diversity {1.2}--------------
阳光不照临，劳荤姒涴秀侨灶箱混朗岸脑冢虢诗丹绕丹
------------Diversity {1.6}--------------
香厨流瀑布，翠岛威骆莼袋袿础壮杖皇树靳色鹆绡耐死
Epoch 25/5816
128/128 [==============================] - 45s 352ms/step - loss: 6.7684 - acc: 0.1641

==================Epoch {24}=====================
------------Diversity {0.7}--------------
圆魄上寒空，放辇辰光宇高昔蒨羌飞飞皇四辇孰辙敦玉
------------Diversity {1.2}--------------
早遇圣明朝，形鸭弧募孩荤个追剃缆赢寓弦霜赏挤道形
------------Diversity {1.6}--------------
飞云临紫极，无襟封跳断机薰熙膊绚振辜润写洲痹馀残
Epoch 26/5816
128/128 [==============================] - 45s 350ms/step - loss: 6.9709 - acc: 0.1562

==================Epoch {25}=====================
------------Diversity {0.7}--------------
江州望通州，情缀朗汉露川苑纲耨筵悲交开衡苑黄务禁
------------Diversity {1.2}--------------
幽州胡马客，厥媛鸦质浇川欹景淬淅疑司宫宫猿洏径緺
------------Diversity 

128/128 [==============================] - 45s 350ms/step - loss: 6.3713 - acc: 0.1641

==================Epoch {46}=====================
------------Diversity {0.7}--------------
十里一走马，留眺来上还处寻云玳亡节朝初渚梅去熏戎
------------Diversity {1.2}--------------
生长在荥阳，操卫凝芒辔粪擎楫盖葱肃状溜鸟居起谏亡
------------Diversity {1.6}--------------
寒楚十二月，轩经空婢暂泛未滢潋尽打讴跹可弧锦歆誾
Epoch 48/5816
128/128 [==============================] - 46s 356ms/step - loss: 6.7825 - acc: 0.1719

==================Epoch {47}=====================
------------Diversity {0.7}--------------
闻说融峰下，山尽临潜集累息虚想宾嗣还烽疏多机风举
------------Diversity {1.2}--------------
只贪诗调苦，荒静兽乃鞍粤浅玩禁帷堤扇难囹礼既厉点
------------Diversity {1.6}--------------
绝域长夏晚，咤佛或粤曜百府哉穰膜废迹橡涛褫沤先莲
Epoch 49/5816
128/128 [==============================] - 45s 352ms/step - loss: 6.8146 - acc: 0.1719

==================Epoch {48}=====================
------------Diversity {0.7}--------------
层轩登皎月，雾封里静盖筑奉灭烟汾沙文蔓文怀陈成上
------------Diversity {1.2}--------------
曾梦诸侯笑，重齐戟田然萃峒心钟屿重漠桡籹露寝送蝀
------------Diversity 

128/128 [==============================] - 45s 352ms/step - loss: 6.5013 - acc: 0.1719

==================Epoch {69}=====================
------------Diversity {0.7}--------------
春眠不觉晓，梅林形还方席开笳染兹垂席万叶液一金款
------------Diversity {1.2}--------------
独立身难达，道璇四书启古。只临国平疑抚临流佑轩志
------------Diversity {1.6}--------------
只谓一苍翠，穟霜鉴太缦岁难檐隐寻暇纪续圃况赏色真
Epoch 71/5816
128/128 [==============================] - 45s 353ms/step - loss: 6.5764 - acc: 0.1641

==================Epoch {70}=====================
------------Diversity {0.7}--------------
野鹤一辞笼，傲夜纷里鸣书节尘约月旧馀知宴股舆落入
------------Diversity {1.2}--------------
青山日将暝，独温康神桐聊布原通山今诜程原房皇政银
------------Diversity {1.6}--------------
西郭尘埃外，蛮灭暄庶蜘所勒枝仰滪胜向仰全戍令戈摅
Epoch 72/5816
128/128 [==============================] - 45s 352ms/step - loss: 6.8446 - acc: 0.1719

==================Epoch {71}=====================
------------Diversity {0.7}--------------
高关闲独望，黄回横转树暮一带关梁忘迁得先隆机运罗
------------Diversity {1.2}--------------
上略在安边，知姬迥全今收暂生近谏术雁野衣元璧为荒
------------Diversity 

128/128 [==============================] - 45s 354ms/step - loss: 6.4168 - acc: 0.1719

==================Epoch {92}=====================
------------Diversity {0.7}--------------
一日又将暮，半寻眷方芳时千筵千举升寸待水阳节鳞飞
------------Diversity {1.2}--------------
画君年少时，难凝山将炎引滋城尘穴共桂名聊璧轩务万
------------Diversity {1.6}--------------
新溜满澄陂，褕璧桷广辰回花月超内必若石栖阶频壶摧
Epoch 94/5816
128/128 [==============================] - 45s 349ms/step - loss: 6.4844 - acc: 0.1719

==================Epoch {93}=====================
------------Diversity {0.7}--------------
碧涧虽多雨，歌惜将平尊背渚节琼无横重庭地始琴川雾
------------Diversity {1.2}--------------
极浦三春草，不能吹神烬染夏暌良云冷香难暂想碧山歌
------------Diversity {1.6}--------------
雪点翠云裘，故望兵瀛罘潢暨缀玄本宴蝗否寒徇贤熏倾
Epoch 95/5816
128/128 [==============================] - 45s 355ms/step - loss: 6.5388 - acc: 0.1641

==================Epoch {94}=====================
------------Diversity {0.7}--------------
流水何山分，餐出山殿疏雾意以度无股阿缀芳得积发百
------------Diversity {1.2}--------------
月下留丹灶，宫干司衣千郡双正庆多殉槿汾留危耀涅力
------------Diversity 

...

128/128 [==============================] - 45s 354ms/step - loss: 6.3291 - acc: 0.1875

==================Epoch {5713}=====================
------------Diversity {0.7}--------------
山头戒坛路，松边相有生何何人安不金还向朝处离谁成
------------Diversity {1.2}--------------
一从征战后，神如流不中水溪应落意柳气徒多幽前以同
------------Diversity {1.6}--------------
出送乡人尽，何外音玄连归老青苍仙东知将吟花衣九待
Epoch 5715/5816
128/128 [==============================] - 45s 351ms/step - loss: 8.5057 - acc: 0.1641

==================Epoch {5714}=====================
------------Diversity {0.7}--------------
雁塞日初晴，地去定爱不应石亦同思从谁多在开开转苔
------------Diversity {1.2}--------------
举目伤芜没，乘离远女仙上晓不叶暮命千飞色无上静暮
------------Diversity {1.6}--------------
卖药曾相识，正双绝青士欲忽重神送早寒逢以飞却不未
Epoch 5716/5816
128/128 [==============================] - 45s 355ms/step - loss: 7.7578 - acc: 0.1719

==================Epoch {5715}=====================
------------Diversity {0.7}--------------
双莺衔野蝶，事露所时人云石不桂方如知不见时不为花
------------Diversity {1.2}--------------
天下兵又动，清移出令青竹亦苔去轻者昏途用静雨壁多
------------

128/128 [==============================] - 46s 362ms/step - loss: 6.5874 - acc: 0.1797

==================Epoch {5736}=====================
------------Diversity {0.7}--------------
君王巡海内，分上亦多处人行临枝不重过自为凌落客春
------------Diversity {1.2}--------------
舟小回仍数，金长雪海是步身当山月诗里首仙一独云此
------------Diversity {1.6}--------------
款塞旋征骑，已客晚外见遥朝分许水无空节在照鸟四溪
Epoch 5738/5816
128/128 [==============================] - 46s 360ms/step - loss: 6.8189 - acc: 0.1641

==================Epoch {5737}=====================
------------Diversity {0.7}--------------
驱使鬼神功，非柳生无衣烟泉远如叶过今路不长有当衣
------------Diversity {1.2}--------------
枕上用心静，河头花枝回宫客马头烟万伤当动赋深户何
------------Diversity {1.6}--------------
夜光贮怀袖，宿谷可庭中长时重朱衣岁阳明思曲过多忧
Epoch 5739/5816
128/128 [==============================] - 46s 363ms/step - loss: 8.0703 - acc: 0.1875

==================Epoch {5738}=====================
------------Diversity {0.7}--------------
昔叹谗销骨，名山相情清生还在雨草愿上云秋胡车钟流
------------Diversity {1.2}--------------
巫山望不极，向仙霞见传山昏许林平名泉欢生在云山夜
------------

128/128 [==============================] - 46s 362ms/step - loss: 7.5971 - acc: 0.1719

==================Epoch {5759}=====================
------------Diversity {0.7}--------------
魏王铜雀妓，春影开门如往离此游春山客身来客如别坐
------------Diversity {1.2}--------------
黄雀衔黄花，百落难长至台城夫歌草没同信莫随风自草
------------Diversity {1.6}--------------
一身禅诵苦，留忽应声望飞欢西君夜高不孤车岁江笑意
Epoch 5761/5816
128/128 [==============================] - 46s 360ms/step - loss: 8.0555 - acc: 0.1719

==================Epoch {5760}=====================
------------Diversity {0.7}--------------
儿童戏穿凿，青山人重在雨处红未朝回未蓬叶旧地不如
------------Diversity {1.2}--------------
灯心生众草，水里子秋门知秋仍家坐玉如马阴尘白陵孤
------------Diversity {1.6}--------------
儒衣两少年，出家愁犹易晚能遥闻和载间谷白莫月送子
Epoch 5762/5816
128/128 [==============================] - 46s 361ms/step - loss: 7.8675 - acc: 0.1797

==================Epoch {5761}=====================
------------Diversity {0.7}--------------
极目烟霞外，文知行作分外日声无上处月不见家海夜事
------------Diversity {1.2}--------------
有相无相身，仙声流山道心不寒月花在楚仙头天年情今
------------

128/128 [==============================] - 46s 360ms/step - loss: 7.8993 - acc: 0.1797

==================Epoch {5782}=====================
------------Diversity {0.7}--------------
屡别容华改，心清云经白遥碧车林如不应高叶地水孤风
------------Diversity {1.2}--------------
送别枯桑下，观远怜风已采镜归白清青怀色静云深碧极
------------Diversity {1.6}--------------
楼观岳阳尽，景圣门亦江华日上马深生阳怜五西年城长
Epoch 5784/5816
128/128 [==============================] - 46s 362ms/step - loss: 7.6384 - acc: 0.1719

==================Epoch {5783}=====================
------------Diversity {0.7}--------------
归羡辽东鹤，凤人思云落花见门阴华来一无同亭行何国
------------Diversity {1.2}--------------
汉代簪缨盛，间殊仙云林不出草长古处人树散思微长胡
------------Diversity {1.6}--------------
我来竟何事，身大新阳风新初红影来知馀长游前独秋已
Epoch 5785/5816
128/128 [==============================] - 46s 360ms/step - loss: 7.4917 - acc: 0.1875

==================Epoch {5784}=====================
------------Diversity {0.7}--------------
长大有南北，谁龙风自当东青山石竹已云时万门出因白
------------Diversity {1.2}--------------
斜廊连绮阁，留望云万微素自人非楚重对香为我处庭古
------------

128/128 [==============================] - 46s 357ms/step - loss: 7.8150 - acc: 0.1797

==================Epoch {5805}=====================
------------Diversity {0.7}--------------
家临清溪水，鸟家看在风幽香树应云道心枝相道风高归
------------Diversity {1.2}--------------
栖栖南越鸟，出海亲异无云老动西终玉游军长与然方心
------------Diversity {1.6}--------------
一雨西城色，寄淮观庭落宿青生别偏书梦柳去光何名霜
Epoch 5807/5816
128/128 [==============================] - 46s 362ms/step - loss: 8.1572 - acc: 0.1719

==================Epoch {5806}=====================
------------Diversity {0.7}--------------
独倚秋光岸，高不复异复朝情香何空木中随枕何万仙三
------------Diversity {1.2}--------------
五天何处望，定朝蓬起清积起光诗君若开满冠出知声霜
------------Diversity {1.6}--------------
芳意何能早，休恨江转门秋叶木来客待天通客同宁去惭
Epoch 5808/5816
128/128 [==============================] - 46s 355ms/step - loss: 9.5806 - acc: 0.1641

==================Epoch {5807}=====================
------------Diversity {0.7}--------------
潮来逐潮上，方十里何地一日风年人关幽光在晓开不见
------------Diversity {1.2}--------------
何处成吾道，楼川何独成恨里年远北景皇同将林见安高
------------